### Plot escape magnitudes and IC50 differences for each mutation

In [ ]:
import pandas as pd
import altair as alt
import httpimport
import numpy as np

_ = alt.data_transformers.disable_max_rows()

In [ ]:
# Import custom altair theme from remote github using httpimport module
def import_theme_new():
    with httpimport.github_repo("bblarsen-sci", "altair_themes", "main"):
        import main_theme

        @alt.theme.register("custom_theme", enable=True)
        def custom_theme():
            return main_theme.main_theme()


import_theme_new()

In [ ]:
# input data
# escape data
escape_df = pd.read_csv('../../results/filtered_data/antibody_escape/combined/all_antibodies_escape_filtered.csv')

# IC50 relative validation data
#ic50_df = pd.read_csv('../../data/experimental_data/relative_validation_IC50s.csv')
ic50_df = pd.read_csv(
    "../../results/ic50_data/top_escape_ic50_estimates_merged_with_escape.csv"
)
ic50_df['ic50_fold'] = ic50_df['ic50_fold'].round(0).astype(int)
display(ic50_df)
# parameters
antibody_order = [
    "12B2",
    "2D3",
    "4H3",
    "1A9",
    "1F2",
    "2B12",
]


In [ ]:
# Plot IC50 values for unmutated and mutant for each antibody

# Base chart
ic50_chart_base = alt.Chart(ic50_df).mark_point(
    filled=True, 
    strokeWidth=1, 
    stroke="black", 
    size=150, 
    opacity=1, 
    color='#56B4E9').encode(
    x=alt.X(    
        "antibody:O",
        title=None,
        sort=antibody_order
    )
)

# Unmutated and mutant points
unmutated = ic50_chart_base.encode(y=alt.Y('ic50_unmutated:Q', title=None), color=alt.value('#56B4E9'))
mutant = ic50_chart_base.encode(y=alt.Y('ic50_mutant:Q', title=None), color=alt.value('#E69F00'))

# Gray lines connecting unmutated and mutant points
gray_lines = alt.Chart(ic50_df).mark_rule(
    color='#b8b0ac',
    opacity=0.75
).encode(
    x=alt.X(
        "antibody:O",
        title=None,
        sort=antibody_order,
        axis=alt.Axis(labelAngle=0),
    ),
    y=alt.Y(
        'ic50_unmutated:Q',
        axis=alt.Axis(tickCount=3, grid=False, labelAngle=0),
        scale=alt.Scale(type='log')
    ),
    y2=alt.Y2(
        'ic50_mutant:Q'
    )
)

# Horizontal line at y=10 (limit of detection)
rule = alt.Chart().mark_rule(
    color='#b8b0ac',
    strokeDash=[2, 2],
    opacity=0.5
).encode(
    y=alt.datum(10)
)

# Text labels for fold change
text = (
    alt.Chart(ic50_df)
    .mark_text(dy=-10, baseline="bottom", align="center")
    .encode(
        x=alt.X(
            "antibody:O",
            title=None,
            sort=antibody_order,
            axis=alt.Axis(labelAngle=0),
        ),
        y=alt.datum(10),
        text=alt.Text("ic50_fold:N"),
    )
)

# Combine all layers
combined_IC50_change = alt.layer(
    rule,
    gray_lines,
    unmutated,
    mutant,
    text
).properties(
    width=300,
    height=200)

display(combined_IC50_change)
#combined_IC50_change.save("../../ic50_chart.png", ppi=300)
#combined_IC50_change.save("../../ic50_chart.svg")

In [ ]:
# Make jitter plot of escape data for each antibody

customColors = [
    "#5778a4",
    "#e49444",
    "#d1615d",
    "#85b6b2",
    "#6a9f58",
    "#e7ca60",
    "#a87c9f",
    "#f1a2a9",
    "#967662",
    "#b8b0ac",
]

escape_magnitudes = (
    alt.Chart(escape_df.query("escape_mean > 0.1"))
    .mark_point(filled=True, strokeWidth=0.75, stroke="black", size=75, opacity=1)
    .encode(
        x=alt.X(
            "antibody:N",
            title=None,
            sort=antibody_order,
            bandPosition=0.5,
            axis=alt.Axis(labelAngle=0),
        ),
        y=alt.Y("escape_mean:Q", title=None, axis=alt.Axis(tickCount=2, grid=True)),
        xOffset=alt.XOffset("jitter:Q", bandPosition=0.5),
        color=alt.Color(
            "antibody:N",
            title=None,
            scale=alt.Scale(domain=antibody_order, range=customColors),
            legend=None,
        ),
        tooltip=["antibody", "escape_mean"],
    )
    .transform_calculate(jitter="sqrt(-2*log(random()))*cos(2*PI*random())")
)

escape_magnitudes.display()

In [ ]:
## Plot total escape for each antibody
sum_escape_df = escape_df.groupby(['antibody', 'site']).agg(
    total_escape = ('escape_mean', 'sum')
).reset_index()
display(sum_escape_df)

escape_magnitudes_sum = (
    alt.Chart(sum_escape_df.query("total_escape > 0.1"))
    .mark_point(filled=True, strokeWidth=0.75, stroke="black", size=75, opacity=1)
    .encode(
        x=alt.X(
            "antibody:N",
            title=None,
            sort=antibody_order,
            bandPosition=0.5,
            axis=alt.Axis(labelAngle=0),
        ),
        y=alt.Y("total_escape:Q", title=None, axis=alt.Axis(tickCount=2, grid=True)),
        xOffset=alt.XOffset("jitter:Q", bandPosition=0.5),
        color=alt.Color(
            "antibody:N",
            title=None,
            scale=alt.Scale(domain=antibody_order, range=customColors),
            legend=None,
        ),
        tooltip=["antibody", "total_escape"],
    )
    .transform_calculate(jitter="sqrt(-2*log(random()))*cos(2*PI*random())")
)

escape_magnitudes_sum.display()

### Plot functional effects of mutations in antibody footprint

In [ ]:
### load interface func effects
distances_df = pd.read_csv('../../results/atomic_distances/combined_distances.csv').drop(columns=['f_chain','mab_site','mab_residue','mab_chain'])
entry_df = pd.read_csv('../../results/filtered_data/cell_entry/Nipah_F_func_effects_filtered_mean.csv')
display(distances_df, entry_df)

In [ ]:
# first get the unique antibodies in the escape data
min_dist_cutoff = 5
unique_antibodies_list = escape_df['antibody'].unique().tolist()
print(f"Unique antibodies in escape data: {unique_antibodies_list}")

# iterate through each antibody and get the sites that are within the min_dist_cutoff
# and then get the effects of those sites from the entry dataframe
# this will create a new dataframe with the effects of the close sites for each antibody
# then merge this dataframe with the escape dataframe to get the escape effects for those sites
empty = []
for antibody in unique_antibodies_list:
    tmp_df = distances_df.query(f'antibody == "{antibody}" and min_distance < {min_dist_cutoff}')
    close_sites = tmp_df['site'].unique().tolist()
    tmp_effect_df = entry_df[entry_df['site'].isin(close_sites)].assign(antibody=antibody)
    empty.append(tmp_effect_df)
close_sites_effects_df = pd.concat(empty, ignore_index=True)

# merge the close sites effects dataframe with the escape dataframe to get the escape effects for those sites
display(close_sites_effects_df)


In [ ]:
# Make boxplot of functional effects of mutations in antibody footprint
chart_boxplot = (
    alt.Chart(close_sites_effects_df)
    .mark_boxplot(extent="min-max", opacity=1)
    .encode(
        x=alt.X(
            "antibody:N", title=None, sort=antibody_order, axis=alt.Axis(labelAngle=0)
        ),
        y=alt.Y("effect:Q", title=None),
        color=alt.Color(
            "antibody:N",
            legend=None,
            scale=alt.Scale(domain=antibody_order, range=customColors),
        ),
    )
)
chart_boxplot.display()


### Now combine the charts 

In [ ]:
concat_chart = alt.vconcat(
    escape_magnitudes.properties(height=125, width=alt.Step(40)),
    combined_IC50_change.properties(height=125),
    escape_magnitudes_sum.properties(height=125),
    chart_boxplot.properties(height=125)
).resolve_scale(x="shared")

concat_chart.display()

concat_chart.save('../../results/figures/antibody_escape/nipah_DMS_escape_magnitudes.png', ppi=300)
concat_chart.save('../../results/figures/antibody_escape/nipah_DMS_escape_magnitudes.svg')